Results saved at the same glmdir folders as before correction.

- csdbase.sig.voxel.mgh - the sig volume corrected for multiple
comparisons on a voxel-wise basis. The threshhold and sign are
irrelevant. The value at each voxel is the corrected -log10(p-value)
for that voxel.

- csdbase.sig.cluster.mgh - the sig volume corrected for multiple
comparisons on a cluster-wise basis. The value at each voxel
is the -log10(p), where p is the pvalue of the cluster at
that voxel. If that voxel does not belong to a cluster, its
value will be 0.

- csdbase.sig.cluster.summary - this is the cluster summary table
(a simple text file).

- csdbase.y.ocn.dat - this is a summary of the input (y) over each
cluster. It has a column for each cluster. Each row is a subject. The
value is the average of the input (y) in each cluster. This is a
simple text file.

- csdbase.sig.ocn.mgh - output cluster number volume. The value of the
voxel is the integer cluster number that that voxel belongs to. This
can be used like a segmentation (eg, to load into tkmedit or
use for running mri_segstats).

- csdbase.sig.ocn.annot (for surfaces only) - this is an annotation
with the annotation name being the cluster number. This can be
used like any other annotation (eg, to load into tksurfer or
as input for mri_segstats or mris_anatomical_stats).csdbase.sig.masked.mgh - the original sig volume masked to show
only clusters.

In [ ]:
import numpy as np
import pandas as pd
import os
import subprocess
import shutil
from pathlib import Path

def setup_freesurfer():
    """Initialize FreeSurfer environment"""
    freesurfer_home = '/home/localadmin/freesurfer'
    subjects_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia"
    
    os.environ['FREESURFER_HOME'] = freesurfer_home
    os.environ['SUBJECTS_DIR'] = subjects_dir
    
    setup_cmd = f"bash -c 'source {freesurfer_home}/SetUpFreeSurfer.sh; env'"
    try:
        process = subprocess.Popen(setup_cmd, stdout=subprocess.PIPE, shell=True)
        output, _ = process.communicate()
        
        for line in output.decode().split('\n'):
            if '=' in line:
                key, value = line.split('=', 1)
                os.environ[key] = value
        
        print(f"FreeSurfer environment initialized successfully")
        print(f"FREESURFER_HOME: {os.environ.get('FREESURFER_HOME')}")
        print(f"SUBJECTS_DIR: {os.environ.get('SUBJECTS_DIR')}")  # Add this line
    except Exception as e:
        print(f"Error setting up FreeSurfer: {e}")
        raise
    
def run_glmfit_sim(glmdir):
    # Get base folder name
    folder_name = Path(glmdir).name
    
    # For group_results
    if 'group_results' in str(glmdir):
        contrast = 'group_contrast_' + folder_name.split('demeaned_')[-1].replace('.glmdir', '')
    # For metabolite results (glu/gsh/ins/tcho)
    else:
        contrast = folder_name.split('demeaned_')[-1].replace('.glmdir', '')

    sig_path = os.path.join(glmdir, contrast, 'sig.mgh')
    print(f"Looking for sig.mgh at: {sig_path}")
    
    if not os.path.exists(sig_path):
        print(f"sig.mgh not found at {sig_path}")
        return False
        
    cmd = [
        'mri_glmfit-sim',
        '--glmdir', glmdir,
        '--perm', '1000', '2', 'abs',  # permutation simulation with 5000 iterations, threshold=2 (-log10(p)=0.01), abs=both pos/neg
        '--cwp', '0.05', # cluster-wise p-value threshold
        # '--2spaces',  # for bilateral surface analysis
        '--bg', '1'
    ]
    print(' '.join(cmd))
    try:
        result = subprocess.run(
            ' '.join(cmd), 
            shell=True, 
            check=True, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE, 
            text=True
        )
        print(f"Success for {glmdir}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error for {glmdir}:")
        print(e.stderr)
        return False
        

    
setup_freesurfer()

result_dirs = [
    "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_results"
]

for results_dir in result_dirs:
    print(f"\nProcessing {results_dir}")
    # Get all .glmdir folders
    glmdirs = [d for d in os.listdir(results_dir) if d.endswith('.glmdir')]
    
    for glmdir in glmdirs:
        full_path = os.path.join(results_dir, glmdir)
        if os.path.isdir(full_path):
            run_glmfit_sim(full_path)